<div style="border: 1px solid white; padding: 5px; margin-right: auto;  width: 80%;">

✍ В предыдущих разделах мы собирали полезную информацию с различных сайтов.

Вы могли убедиться, что поиск необходимой информации с выделением правильных тегов — довольно трудоёмкая задача. Кроме того, подобные программы могут ломаться в случаях, когда меняется дизайн сайта, его разметка или владельцы сайтов защищаются от ботов капчей.
</div>

К счастью, многие крупные сайты предоставляют доступ к так называемым API (англ. Application Programming Interface, рус. Интерфейс Прикладного Программирования).

<div style="background-color: #e0fff3; padding: 15px; color: black; width: 80%;">  <b>API</b> — это специальные разделы сайта, где информацию можно получать без разметки, а формат запросов и ответов зафиксирован. API созданы для того, чтобы облегчить взаимодействие с сайтом для сторонних разработчиков.</div>

К примеру, мы уже видели, как ресурс Курсы валют ЦБ РФ в XML и JSON возвращает данные о валютах в JSON-формате. Это пример API.

Рассмотрим на примере социальной сети ВКонтакте особенности API, характерные для более крупных сайтов.

## Ключ авторизации

Для того чтобы начать работать с API, обычно необходимо получить сервисный ключ авторизации — <b>токен</b>.

<div style="background-color: #e0fff3; padding: 15px; color: black; width: 80%;">  <b>Токен</b> — это средство идентификации пользователя или отдельного сеанса работы в компьютерных сетях и приложениях. Различают <b>программные</b> и <b>аппаратные</b> токены.
Мы будем использовать программный токен, который обычно представляет собой зашифрованную последовательность символов, позволяющую точно идентифицировать объект и определить уровень его привилегий. Он генерируется системой авторизации и привязывается к конкретному сеансу работы, клиенту сети или пакету данных.</div>

Авторизация применяется практически во всех API, чтобы отдавать данные только их владельцу или контролировать количество запросов в единицу времени.

Сервисный токен для API ВКонтакте для нашей задачи создаётся вместе с новым приложением. Приложение мы делать, конечно, не будем. Оно нужно только для получения токена, чтобы сделать необходимые выгрузки.

Зайдите на <a href="https://vk.com/login?u=2&to=L2VkaXRhcHA/YWN0PWNyZWF0ZQ--">страницу</a>, чтобы создать приложение (вы должны быть авторизованы ВКонтакте). Дайте приложению любое название и в разделе Платформа поставьте отметку выбора напротив значения Standalone-приложение:

Подтвердив создание приложения на сайте в приложении ВКонтакте или по СМС, зайдите в Настройки. Нужный нам токен лежит в поле Сервисный ключ доступа:

<img src='../static/img/api_6.jpg'>

## Первые запросы к API

Чтобы познакомиться с работой API, мы будем получать данные для статистических отчётов произвольной группы, например данные о соотношении мужчин и женщин, статистику географии пользователей и т. п.

Сначала рассмотрим работу API на простом примере, на основе которого работают многие системы.

Сделаем наш первый запрос из браузера.

Перейдите по следующей ниже ссылке в браузере, подставив вместо слова TOKEN ваш персональный сервисный ключ доступа (токен), полученный на предыдущем шаге:

https://api.vk.com/method/users.get?user_id=1&v=5.95&access_token=TOKEN

Результат: response	
0	
id	1
first_name	"Pavel"
last_name	"Durov"
can_access_closed	true
is_closed	false

Итак, мы сделали GET-запрос к API ВКонтакте, который состоит из следующих элементов:

- https://api.vk.com/method — домен и URL запроса API; обычно не меняется;
- users.get — название метода, который отдаёт определённый отчёт, в нашем случае это метод для получения информации о пользователе;
- user_id и v — параметры запроса: идентификатор пользователя, о котором хотим получить информацию (в нашем примере мы запрашиваем информацию о первом пользователе), и номер версии API;
- token — токен, который выдаётся только пользователям, имеющим право просматривать определённые данные, например показания счётчиков Яндекс.Метрики вашего проекта; на все остальные запросы без корректного токена система отвечает отказом.

Если мы обратимся к <a href="https://vk.com/dev/users.get">документации метода users.get</a>, то увидим, что в ней описано множество других параметров, которые можно получить о пользователе (дата рождения, пол, родной город и другие) — словом, всё то, что мы видим на странице пользователя в интерфейсе или приложении ВКонтакте (конечно, если пользователь их указал).

Добавим к запросу дату рождения и пол (согласно документации, эти параметры надо перечислять в поле fields):

https://api.vk.com/method/users.get?user_id=1&v=5.95&fields=sex,bdate&access_token=TOKEN

Примечание: значение 2 у параметра sex означает мужской пол.

До этого момента мы делали запросы в браузере — теперь давайте выполним запросы из кода.

## Запрос к API из кода

Продолжаем пользоваться всё той же библиотекой requests.

In [21]:
import requests  # Импортируем модуль requests
from environs import Env

env = Env()
env.read_env(
    "../my.env"
)  ## Прочитать токен с локального файла Environs, где указыван свой сервисный токен

token = env("VK_TOKEN")  # считываем в переменную
url = "https://api.vk.com/method/users.get"  # Указываем адрес страницы к которой делаем запрос
params = {
    "user_id": 1,
    "v": 5.95,
    "fields": "sex,bdate",
    "access_token": token,
    "lang": "ru",
}  # Перечисляем параметры нашего запроса в словаре params
response = requests.get(url, params=params)  # Отправляем запрос
print(response.text)  # Выводим текст ответа на экран

{"response":[{"id":1,"bdate":"10.10.1984","sex":2,"first_name":"Павел","last_name":"Дуров","can_access_closed":true,"is_closed":false}]}


Мы получили строку в JSON-формате, которую можно преобразовать в словарь с помощью метода json(), после чего можно с лёгкостью обращаться к различным полям.

Словари нагляднее выводить с помощью функции pprint(), которую мы уже использовали ранее:

In [22]:
from pprint import pprint  # Импортируем функцию pprint()

pprint(response.json())  # Выводим содержимое словаря, содержащего ответ, на экран

{'response': [{'bdate': '10.10.1984',
               'can_access_closed': True,
               'first_name': 'Павел',
               'id': 1,
               'is_closed': False,
               'last_name': 'Дуров',
               'sex': 2}]}


Как вы видите, по ключу response мы можем получить список, в котором хранятся словари, содержащие информацию о запрошенных нами пользователях. Мы запросили информацию лишь об одном из них, поэтому список содержит только один элемент. Извлечём его:

In [23]:
user = response.json()["response"][
    0
]  # Извлекаем из словаря по ключу response информацию о первом пользователе
print(user["bdate"])  # Выводим дату рождения первого пользователя на экран

10.10.1984


Метод users.get() позволяет запрашивать информацию о множестве (до 1 000) пользователей одновременно. Для этого нужно использовать параметр user_ids и передавать id через запятую в строковом формате. Например, чтобы получить информацию о пользователях с id=1, id=2, id=3, необходимо передать значение параметра user_ids='1,2,3'.

Попробуем это сделать:

In [24]:
ids = ",".join(
    map(str, range(1, 4))
)  # Формируем строку, содержащую информацию о поле id первых трёх пользователей
params = {
    "user_ids": ids,
    "v": 5.95,
    "fields": "bdate",
    "access_token": token,
    "lang": "ru",
}  # Формируем строку параметров
pprint(
    requests.get(url, params=params).json()
)  # Посылаем запрос, полученный ответ в формате JSON-строки преобразуем в словарь и выводим на экран его содержимое, используя функцию pprint()

{'response': [{'bdate': '10.10.1984',
               'can_access_closed': True,
               'first_name': 'Павел',
               'id': 1,
               'is_closed': False,
               'last_name': 'Дуров'},
              {'bdate': '14.2',
               'can_access_closed': False,
               'first_name': 'Александра',
               'id': 2,
               'is_closed': True,
               'last_name': 'Владимирова'},
              {'can_access_closed': True,
               'deactivated': 'deleted',
               'first_name': 'DELETED',
               'id': 3,
               'is_closed': False,
               'last_name': ''}]}


###  Задание 6.2

Используя API, определите долю женщин (sex=1) среди пользователей с id от 1 до 500. Иногда будут попадаться пользователи, у которых пол не указан (sex=0), — таких пользователей не нужно учитывать в общем числе.

В ответе укажите число, округлив до двух знаков после точки-разделителя, например, 0.55.
<div style="background-color: #e0ffd1;color: black;border: 3px solid black; padding: 15px; margin-right: 500px; width: 80%;">
Пример: если у нас будет 300 пользователей с sex=1, 100 пользователей с sex=2 и 100 пользователей с sex=0, то в ответе должно быть 0.75.
</div>
<details>
<summary><strong>Show answer</strong> (Click Here)</summary>
    &emsp; &emsp; <code>
0.49
</code>
</details>

In [27]:
ids = ",".join(
    map(str, range(1, 500))
)  # Формируем строку, содержащую информацию о поле id первых трёх пользователей
params = {
    "user_ids": ids,
    "v": 5.95,
    "fields": "sex",
    "access_token": token,
    "lang": "ru",
}  # Формируем строку параметров
response = requests.get(
    url, params=params
).json()  # Посылаем запрос, полученный ответ в формате JSON-строки преобразуем в словарь и выводим на экран его содержимое, используя функцию pprint()

In [30]:
from collections import Counter

list_of_sex = [x["sex"] for x in response["response"]]
counts = Counter(list_of_sex)
counts

Counter({2: 216, 1: 206, 0: 77})

In [33]:
counts[1] / (counts[1] + counts[2])

0.4881516587677725

## Сбор информации из групп

В одном из предыдущих юнитов в качестве примера мы собрали информацию о небольшом количестве пользователей. Теперь перейдём к более реальной задаче — сбору данных о пользователях группы ВКонтакте.


<div style="background-color: #f5f5f5; padding: 15px; color: black; width: 80%;">
Стоит отметить, что есть много сервисов, которые выгружают похожую статистику из соцсетей. Однако им свойственны недостатки универсальных решений:

- не учитываются все особенности вашего проекта;
- используется фиксированный набор метрик, дополнительную обработку данных приходится делать вам;
- не всегда бесплатны и вряд ли позволят работать с большими объёмами данных.

</div>


<div style="border: 1px solid white; padding: 5px; margin-right: auto;  width: 80%;">
Теперь мы научимся считать произвольные метрики групп, собирая данные из API и работая с двумя ограничениями, которые свойственны практически всем системам:

- ограничение на количество вызовов в единицу времени;
- ограничение на количество выгружаемых строк за один запрос.
</div>


Ограничение на количество запросов в единицу времени сделано для того, чтобы избежать чрезмерной нагрузки на серверы системы. В ряде случаев небольшое количество отчётов можно выгрузить и уложиться в действующий лимит.

Однако второе ограничение не удастся обойти в случае выгрузки больших отчётов. Например, чтобы получить список всех пользователей популярной группы, серверу, возможно, придётся отправить ответ, содержащий миллионы записей.

---

Давайте рассмотрим, как работать с этими ограничениями на примере выгрузки списка пользователей группы https://vk.com/vk социальной сети ВКонтакте.

Обратимся к <a href="https://vk.com/dev/groups">документации</a>, чтобы узнать, какие методы нам доступны для групп, — для получения списка пользователей группы доступен метод <a href="https://vk.com/dev/groups.getMembers">groups.getMembers</a>.

Согласно документации, обязательным параметром данного метода является group_id — идентификатор, или короткое имя, группы. В нашем случае это vk: https://vk.com/vk. Протестируем, как работает метод в самом простом случае, — получим id участников группы:

In [35]:
import requests  # Импортируем модуль requests

token = env("VK_TOKEN")
url = "https://api.vk.com/method/groups.getMembers"  # Указываем адрес обращения
params = {
    "group_id": "vk",
    "v": 5.95,
    "access_token": token,
}  # Формируем строку параметров
response = requests.get(url, params=params)  # Посылаем запрос
data = response.json()  # Ответ сохраняем в переменной data в формате словаря
print(data)  # Выводим содержимое переменной data на экран (отображён фрагмент)

{'response': {'count': 13377032, 'items': [6, 19, 47, 54, 79, 177, 198, 212, 219, 239, 243, 345, 407, 421, 450, 467, 485, 510, 550, 619, 640, 690, 702, 721, 804, 809, 831, 832, 834, 847, 900, 905, 907, 914, 930, 943, 952, 958, 966, 976, 979, 997, 1000, 1018, 1023, 1032, 1033, 1038, 1039, 1059, 1097, 1131, 1139, 1140, 1159, 1174, 1185, 1188, 1301, 1333, 1334, 1351, 1381, 1386, 1388, 1406, 1411, 1418, 1432, 1494, 1503, 1531, 1550, 1568, 1586, 1590, 1593, 1598, 1610, 1615, 1632, 1634, 1650, 1679, 1690, 1697, 1698, 1699, 1700, 1721, 1740, 1754, 1796, 1814, 1820, 1829, 1834, 1839, 1840, 1843, 1858, 1863, 1869, 1887, 1889, 1917, 1943, 1947, 1955, 1969, 2019, 2028, 2050, 2051, 2052, 2059, 2077, 2103, 2136, 2150, 2195, 2201, 2230, 2236, 2273, 2281, 2296, 2298, 2376, 2389, 2395, 2403, 2412, 2436, 2456, 2466, 2470, 2484, 2512, 2513, 2515, 2539, 2571, 2576, 2592, 2601, 2622, 2644, 2654, 2692, 2706, 2745, 2755, 2767, 2787, 2797, 2827, 2858, 2909, 2919, 2922, 2932, 2953, 2962, 2982, 3001, 3006, 304

По ключу count мы можем получить общее число участников группы, а список по ключу items хранит их id. Посмотрим на него поближе:

In [36]:
print(len(data["response"]["items"]))  # Выводим на экран количество элементов словаря

1000


Мы видим, что всего пользователей в группе больше 11 миллионов, а получили мы только первую тысячу пользователей группы. По информации, указанной в документации о параметре count, это максимум, который может отдать API за один раз.

Для получения следующей тысячи пользователей можно воспользоваться параметром offset (с англ. смещение), который передвинет начало отсчёта. Для выгрузки всех пользователей группы будем в цикле выгружать по 1000 пользователей (count будет всегда равен 1000), увеличивая смещение offset на величину count.

Для тренировки напишем цикл выгрузки первых 20 пользователей со значением count=5. Иными словами, мы будем выгружать по пять пользователей за запрос до тех пор, пока не получим информацию о 20 пользователях.

Давайте выведем на экран первые 20 пользователей из нашей первой попытки получить информацию о 1000 пользователей, чтобы мы могли сверить результат выгрузки из 20 пользователей:

In [37]:
users_for_checking = data["response"]["items"][
    :20
]  # Загружаем в переменную информацию об id первых 20 пользователей в виде списка
print(users_for_checking)  # Выводим перечень id первых 20 пользователей

[6, 19, 47, 54, 79, 177, 198, 212, 219, 239, 243, 345, 407, 421, 450, 467, 485, 510, 550, 619]


Теперь используем count и offset, чтобы получить те же id по пять за раз:

In [38]:
url = "https://api.vk.com/method/groups.getMembers"  # Указываем адрес обращения
count = 5
offset = 0
user_ids = []
max_count = 20
while offset < max_count:
    # Будем выгружать по count=5 пользователей,
    # начиная с того места, где закончили на предыдущей итерации (offset)
    print("Выгружаю {} пользователей с offset = {}".format(count, offset))
    params = {
        "group_id": "vk",
        "v": 5.95,
        "count": count,
        "offset": offset,
        "access_token": token,
    }
    response = requests.get(url, params=params)
    data = response.json()
    user_ids += data["response"]["items"]
    # Увеличиваем смещение на количество строк, которое мы уже выгрузили
    offset += count
print(user_ids)

Выгружаю 5 пользователей с offset = 0
Выгружаю 5 пользователей с offset = 5
Выгружаю 5 пользователей с offset = 10
Выгружаю 5 пользователей с offset = 15
[6, 19, 47, 54, 79, 177, 198, 212, 219, 239, 243, 345, 407, 421, 450, 467, 485, 510, 550, 619]


Сравним списки, полученные двумя способами:

In [39]:
print(user_ids == users_for_checking)

True


Так как результат сравнения — True, списки идентичны. Значит, второй способ работает корректно. Теперь мы можем получить данные обо всех пользователях, выставив count = 1000 и max_count = data['response']['count'].

## Ограничение по частоте запросов

<div style="background-color: #f5f5f5; padding: 15px; color: black; width: 80%;">→ В API часто добавляют ограничение по частоте запросов, чтобы отдельно взятые пользователи слишком сильно не перегружали сервер. Подобное ограничение есть и у ВКонтакте — в документации указано, что можно делать не более трёх запросов в секунду.</div>

Чтобы не следить за частотой отправки запросов с секундомером в руках, мы можем после каждого запроса делать паузу. В этом случае, даже если код будет выполняться на самом быстром компьютере, мы не нарушим установленное ограничение, так как периодичность отправки запросов будет искусственно замедлена.

Воспользуемся библиотекой time и методом sleep, с помощью которого мы можем добавить паузу, например в 0.5 секунд, после каждого запроса:

In [40]:
import requests  # Импортируем модуль requests
import time  # Импортируем модуль time

token = env("VK_TOKEN")
url = "https://api.vk.com/method/groups.getMembers"  # Указываем адрес страницы, к которой делаем запрос
count = 1000
offset = 0
user_ids = []
while offset < 5000:
    params = {
        "group_id": "vk",
        "v": 5.95,
        "count": count,
        "offset": offset,
        "access_token": token,
    }
    response = requests.get(url, params=params)
    data = response.json()
    user_ids += data["response"]["items"]
    offset += count
    print("Ожидаю 0.5 секунды...")
    time.sleep(0.5)
print("Цикл завершен, offset =", offset)

Ожидаю 0.5 секунды...
Ожидаю 0.5 секунды...
Ожидаю 0.5 секунды...
Ожидаю 0.5 секунды...
Ожидаю 0.5 секунды...
Цикл завершен, offset = 5000


## Лайки, репосты и комментарии

Через API новостной ленты ВКонтакте мы можем получить информацию о взаимодействии с сообщениями в ленте.

Для примера продолжим работать с группой https://vk.com/vk и рассмотрим последние 100 сообщений в новостной ленте.

<div style="background-color: #e0ffd1;color: black;border: 3px solid black; padding: 15px; margin-right: 500px; width: 80%;">Примечание: обратите внимание, что т.к. сообщения в новостной ленте непрерывно обновляются, то ваш результат выполнения кода ниже будет отличаться от нашего варианта.</div>

Для получения информации о сообщениях на стене в API ВКонтакте предусмотрен метод <a href="https://vk.com/dev/wall.get">wall.get</a>. Применим его:

In [41]:
import requests  # Импортируем модуль requests
from pprint import pprint  # Импортируем функцию pprint()

token = env("VK_TOKEN")
url = "https://api.vk.com/method/wall.get"  # Указываем адрес страницы, к которой делаем запрос
params = {
    "domain": "vk",
    "filter": "owner",
    "count": 1000,
    "offset": 0,
    "access_token": token,
    "v": 5.95,
}
response = requests.get(url, params=params)
pprint(response.json())

{'response': {'count': 645,
              'items': [{'attachments': [{'photo': {'access_key': 'f151a65169472dd7eb',
                                                    'album_id': -7,
                                                    'date': 1698669687,
                                                    'id': 457344737,
                                                    'owner_id': -22822305,
                                                    'sizes': [{'height': 66,
                                                               'type': 's',
                                                               'url': 'https://sun7-21.userapi.com/impg/mWB54mr_Nd7_W09DBQpHGrSpejl2F0K89RPGKw/zYnQ010-B34.jpg?size=75x66&quality=95&sign=3111961d909d1ff44b6bbccb71dedbec&c_uniq_tag=qZqgUGj9RKuzBOWpfE5lhTUefnWj1UsnIeHgWUgg5eM&type=album',
                                                               'width': 75},
                                                              {'height': 114,
     

Посмотрим на количество результатов:

In [42]:
len(response.json()["response"]["items"])
## 100

100

Посмотрим на информацию об отдельном сообщении:

In [43]:
response.json()["response"]["items"][0]

{'inner_type': 'wall_wallpost',
 'comments': {'can_post': 1, 'count': 921, 'groups_can_post': True},
 'marked_as_ads': 0,
 'hash': 'BwkiBSZYk1lHf_jBtw',
 'type': 'post',
 'attachments': [{'type': 'photo',
   'photo': {'album_id': -7,
    'date': 1698669687,
    'id': 457344737,
    'owner_id': -22822305,
    'access_key': 'f151a65169472dd7eb',
    'sizes': [{'height': 66,
      'type': 's',
      'width': 75,
      'url': 'https://sun7-21.userapi.com/impg/mWB54mr_Nd7_W09DBQpHGrSpejl2F0K89RPGKw/zYnQ010-B34.jpg?size=75x66&quality=95&sign=3111961d909d1ff44b6bbccb71dedbec&c_uniq_tag=qZqgUGj9RKuzBOWpfE5lhTUefnWj1UsnIeHgWUgg5eM&type=album'},
     {'height': 114,
      'type': 'm',
      'width': 130,
      'url': 'https://sun7-21.userapi.com/impg/mWB54mr_Nd7_W09DBQpHGrSpejl2F0K89RPGKw/zYnQ010-B34.jpg?size=130x114&quality=95&sign=bc28e7df35cac6743703cffd2d83a8ff&c_uniq_tag=q_Ym44W7DjBvOpOZv6fzMBFT06WYFyLZz4j7QfXGEaQ&type=album'},
     {'height': 531,
      'type': 'x',
      'width': 604,
   

В полях comments, likes и reposts содержится статистика по взаимодействию с сообщением пользователей (на момент получения информации) — число комментариев, лайков и репостов.

---

Давайте соберём итоговую статистику для последних десяти непустых сообщений в словарь stats. В качестве ключа будем использовать начало сообщения (если начало сообщения пустое, то информацию о таком сообщении проигнорируем), в качестве значения — список с тремя интересующими нас метриками и временем публикации (комментарии, лайки, репосты, дата публикации):

In [44]:
stats = {}
count_post = 0  # Счётчик «непустых» сообщений
for record in response.json()["response"]["items"][:]:
    title = record["text"][:30]
    if title:
        stats[title] = [
            record["comments"]["count"],
            record["likes"]["count"],
            record["reposts"]["count"],
            record["date"],
        ]
        count_post += 1
    if count_post < 10:
        continue
    else:
        break
pprint(stats)

{'15 октября прошёл Международны': [287, 1461, 297, 1697464612],
 'VK Звонки — это удобный инстру': [163, 764, 209, 1697013540],
 'Актуализируйте список друзей —': [921, 908, 413, 1698763005],
 'ВКонтакте обновляет процесс вх': [572, 1164, 584, 1697614588],
 'Вчера мы отметили день рождени': [480, 4987, 133, 1697035986],
 'Делимся добрыми новостями. Нак': [394, 700, 42, 1698338505],
 'Не знаете, что послушать? В VK': [314, 409, 31, 1698056317],
 'Продолжаем делиться историями ': [443, 569, 155, 1697625061],
 'Рисуем граффити весь день — Ёл': [168, 607, 53, 1696935521],
 'Фасад кинотеатра «Октябрь» сег': [127, 989, 61, 1696946297]}


Мы рассмотрели базовое взаимодействие с пользователями и группами. ВКонтакте предоставляет достаточно широкие возможности в своём API: всё, что можно делать вручную через браузер, доступно и в API.

<div style="border: 3px dotted white; padding: 5px; margin-right: auto;  width: 80%;"> 

ДОПОЛНИТЕЛЬНО

Если вы размещаете рекламу во ВКонтакте, то можно выгружать всю статистику через <a href="https://vk.com/dev/ads">ads API</a>.

Полный список методов ВКонтакте можно посмотреть в <a href="https://vk.com/dev/methods">документации</a>.
</div>

## Другие API

Вы познакомились с интерфейсами прикладного программирования — API (на примере API социальной сети ВКонтакте).

API для разработчиков предоставляют и многие другие платформы. Вот список, пожалуй, самых популярных из них:

- <a href="https://developers.google.com/maps/">Google Maps API</a>
- <a href="https://developers.google.com/youtube/">YouTube API</a>
- <a href="https://developer.twitter.com/en/docs">Twitter API</a>
- <a href="https://developers.facebook.com/docs/">Facebook API</a>

Вы также можете воспользоваться интернет-поиском, указав в поисковой строке, например, «курсы валют API» или «прогноз погоды api», — среди первых результатов выдачи чаще всего с лёгкостью можно найти ссылки на необходимый функционал.
